In [1]:
import pandas as pd
from os import path, environ, makedirs
from dotenv import load_dotenv
from unidecode import unidecode

In [ ]:
load_dotenv()

# Carregando os dados extraídos no notebook anterior

Neste notebook, vamos utilizar os dados extraídos e salvos pelo notebook `03 habitação - extração.ipynb`.

In [3]:
input_dir = path.join('dados', 'urbanismo')

In [ ]:
filename = path.join(input_dir, 'orcamento_habitacao_original.csv')
df_orcamento = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='utf8')
df_orcamento

In [ ]:
filename = path.join(input_dir, 'orcamento_regionalizado_habitacao_original.csv')
df_orcamento_r = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='utf8')
df_orcamento_r

In [ ]:
filename = path.join(input_dir, 'pdm_meta_12_original.csv')
df_meta_12 = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='utf8')
df_meta_12

In [ ]:
filename = path.join(input_dir, 'his_entregue_original.csv')
df_his = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='utf8')
df_his

In [ ]:
filename = path.join(input_dir, 'tpu_emitido_original.csv')
df_tpu = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='utf8')
df_tpu

# Transformação e padronização

Nos indicadores de habitação, apenas o ano e as subprefeituras são presentes em vários arquivos. Mas, além disso, os arquivos precisarão de tratamentos específicos. Vamos começar com os mais simples, onde é necessário apenas padronizar as subprefeituras.

Primeiro, vamos carregar os dados de subprefeituras que serão utilizados no Qlik Sense.

## CSV de Subprefeituras do Qlik

In [ ]:
url_subs = environ.get('CSV_SUBPREFEITURAS_QLIK')
df_subs = pd.read_csv(url_subs)
df_subs

In [ ]:
df_subs = df_subs[['sub.CODIGO', 'sub.NOME']]
df_subs

## Chave composta subprefeitura-ano

Como 3 tabelas possuem valores para mais de um ano, também vale a pena a criação de uma chave composta entre subprefeitura e ano. A tabela que possui mais períodos é a tabela da meta 12 do Programa de Metas, com os anos de 2021, 2022, 2023 e 2024. Vamos criar uma tabela com o produto cartesiano entre subprefeituras e anos.

In [ ]:
df_subs_ano = (
    df_subs[['sub.NOME']]
    .merge(pd.Series(data=[2021, 2022, 2023, 2024], name='ano'),
           how='cross')
)

df_subs_ano.loc[:, 'subprefeitura-ano'] = (
    df_subs_ano.loc[:, 'sub.NOME'] + ' | ' + df_subs_ano.loc[:, 'ano'].astype(str)
)

df_subs_ano

## PdM - Meta 12: Prover 49.000 moradias de interesse social

In [ ]:
df_meta_12

In [ ]:
df_meta_12 = df_meta_12.loc[:, ['Subprefeitura', '2021', '2022', '2023', '2024']]

df_meta_12

In [ ]:
df_meta_12 = df_meta_12.melt('Subprefeitura',
                var_name='ano',
                value_name='qtd_unidades_acumulado')

df_meta_12

Como os valores foram divulgados no acumulado entre 2021 e 2024, precisamos ajustar para o incremento de cada ano antes de carregar os dados no Qlik.

In [ ]:
df_meta_12['qtd_unidades'] = df_meta_12.groupby('Subprefeitura')['qtd_unidades_acumulado'].diff()
df_meta_12

In [ ]:
df_meta_12.loc[df_meta_12['ano']=='2021', 'qtd_unidades'] = (
    df_meta_12.loc[df_meta_12['ano']=='2021', 'qtd_unidades_acumulado'])
df_meta_12

In [ ]:
df_meta_12 = df_meta_12.drop(columns='qtd_unidades_acumulado')
df_meta_12

Finalmente, vamos criar uma coluna com os nomes padronizados de subprefeituras.

In [ ]:
subs_meta_12 = df_meta_12['Subprefeitura'].apply(unidecode).unique().tolist()
subs_meta_12.sort()
subs_meta_12

In [ ]:
subs_qlik = df_subs['sub.NOME'].unique().tolist()
subs_qlik.sort()
subs_qlik

In [ ]:
len(subs_meta_12)

Vemos que existem 3 subprefeituras faltantes no dataframe da meta 12. Vamos avaliar quais podem ser. Numa inspeção detalhada vemos que faltam `ARICANDUVA-FORMOSA-CARRAO`, `SAO MIGUEL` e `VILA MARIANA`. Vamos criar uma cópia da lista de subs do qlik adaptada à meta 12.

In [ ]:
subs_qlik_meta_12 = subs_qlik.copy()
subs_qlik_meta_12.remove('ARICANDUVA-FORMOSA-CARRAO')
subs_qlik_meta_12.remove('SAO MIGUEL')
subs_qlik_meta_12.remove('VILA MARIANA')
subs_qlik_meta_12

In [ ]:
mapper_meta_12 = {
    o: q
    for o, q in zip(subs_meta_12, subs_qlik_meta_12)
}

mapper_meta_12

In [ ]:
df_meta_12.insert(1,
                  'sub.NOME',
                  df_meta_12['Subprefeitura'].apply(unidecode).map(mapper_meta_12))
df_meta_12

In [ ]:
df_meta_12['qtd_unidades'] = df_meta_12['qtd_unidades'].astype(int)
df_meta_12['ano'] = df_meta_12['ano'].astype(int)
df_meta_12

In [ ]:
df_meta_12 = df_meta_12.merge(df_subs_ano,
                              how='left',
                              on=['sub.NOME', 'ano'])

df_meta_12

## Produção de habitação de interesse social

In [ ]:
df_his

In [ ]:
subs_his = df_his['região'].apply(unidecode).unique().tolist()
subs_his.sort()
subs_his

In [ ]:
mapper_his = {
    s: q for s, q in zip(subs_his, subs_qlik)
}

mapper_his

In [ ]:
df_his.insert(1,
                  'sub.NOME',
                  df_his['região'].apply(unidecode).map(mapper_his))
df_his

In [ ]:
df_his['qtd_unidades'] = df_his['qtd_unidades'].astype(int)
df_his['ano'] = df_his['ano'].astype(int)
df_his

In [ ]:
df_his = df_his.merge(df_subs_ano,
                              how='left',
                              on=['sub.NOME', 'ano'])

df_his

## Número de termos de Permissão de Uso (TPU) emitidos em nome da mulher da familia

In [ ]:
df_tpu['qtd_termos'] = df_tpu['qtd_termos'].astype(int)
df_tpu['ano'] = df_tpu['ano'].astype(int)
df_tpu

## Orçamento do Programa 3002

Para o orçamento, além de padronizar os nomes de subprefeituras e tipos de dados das métricas, precisaremos também adaptar os dados para compatibilizar o orçamento regionalizado e não realizado. Para isso, vamos fazer o seguinte:

1. Classificar o orçamento detalhado por nível de regionalização nas seguintes categorias: subprefeitura, região e não regionalizável;
1. Agrupar o restante do orçamento não detalhado e manter apenas o orçamento inicial, atualizado e liquidado;
1. Subtrair o total do orçamento detalhado do orçamento não detalhado e classificar o nível de regionalização como não regionalizado;
1. Unir os dois dataframes de orçamento de acordo com as dimensões mantidas.

In [ ]:
df_orcamento_r.head(1)

In [ ]:
cols_orcamento_r = ['CÓDIGO_ÓRGÃO', 'SIGLA_ÓRGÃO', 'DESCRIÇÃO_ÓRGÃO',
                    'CÓDIGO_PROGRAMA', 'DESCRIÇÃO_PROGRAMA',
                    'CÓDIGO_PROJ_ATIV', 'DESCRIÇÃO_PROJ_ATIV', 'REGIÃO',
                    'SUBPREFEITURA', 'TIPO_REGIONALIZAÇÃO']

cols_orcamento_r_vl = ['VALOR_DETALHAMENTO_AÇÃO']

df_orcamento_r = df_orcamento_r[cols_orcamento_r + cols_orcamento_r_vl]
df_orcamento_r

In [ ]:
df_orcamento_r['TIPO_REGIONALIZAÇÃO'].value_counts()

In [ ]:
df_orcamento_r.loc[df_orcamento_r['TIPO_REGIONALIZAÇÃO'].isna(), 'TIPO_REGIONALIZAÇÃO'] = 'Despesa Não-Regionalizável'
df_orcamento_r

In [ ]:
df_orcamento_r['TIPO_REGIONALIZAÇÃO'].value_counts()

In [ ]:
df_orcamento_r = df_orcamento_r.groupby(cols_orcamento_r).sum().round(2).reset_index()

df_orcamento_r

In [ ]:
df_orcamento_r.loc[
    ~df_orcamento_r['REGIÃO'].str.contains('Supra', na=False),
    'NIVEL_REGIONALIZAÇÃO'] = 'Região'

df_orcamento_r

In [ ]:
df_orcamento_r.loc[
    ~df_orcamento_r['SUBPREFEITURA'].str.contains('Supra', na=False),
    'NIVEL_REGIONALIZAÇÃO'] = 'Subprefeitura'

df_orcamento_r

In [ ]:
df_orcamento_r.loc[df_orcamento_r['NIVEL_REGIONALIZAÇÃO'].isna(), 'NIVEL_REGIONALIZAÇÃO'] = 'Não regionalizável'
df_orcamento_r

In [ ]:
df_orcamento.head(1)

In [ ]:
cols_orcamento = ['Cd_Orgao', 'Sigla_Orgao', 'Ds_Orgao', 'Cd_Programa',
                  'Ds_Programa', 'ProjetoAtividade', 'Ds_Projeto_Atividade']

cols_orcamento_vl = ['Vl_Orcado_Ano', 'Vl_Orcado_Atualizado', 'Vl_Liquidado']

df_orcamento = df_orcamento[cols_orcamento + cols_orcamento_vl]
df_orcamento

In [ ]:
df_orcamento = df_orcamento.groupby(cols_orcamento).sum().reset_index()
df_orcamento

In [ ]:
df_orcamento_r_agg = (
    df_orcamento_r[['CÓDIGO_ÓRGÃO', 'CÓDIGO_PROGRAMA',
                    'CÓDIGO_PROJ_ATIV', 'VALOR_DETALHAMENTO_AÇÃO']]
    .groupby(['CÓDIGO_ÓRGÃO', 'CÓDIGO_PROGRAMA', 'CÓDIGO_PROJ_ATIV'])
    .sum()
    .reset_index()
)

df_orcamento_r_agg.loc[:, 'VALOR_DETALHAMENTO_AÇÃO'] = (
    df_orcamento_r_agg
    .loc[:, 'VALOR_DETALHAMENTO_AÇÃO']
    .round(2)
)

df_orcamento_ajustado = df_orcamento.merge(
    df_orcamento_r_agg,
    left_on=['Cd_Orgao', 'Cd_Programa', 'ProjetoAtividade'],
    right_on=['CÓDIGO_ÓRGÃO', 'CÓDIGO_PROGRAMA', 'CÓDIGO_PROJ_ATIV'],
    how='left'
).drop(columns=['CÓDIGO_ÓRGÃO', 'CÓDIGO_PROGRAMA', 'CÓDIGO_PROJ_ATIV'])
df_orcamento_ajustado

In [ ]:
df_orcamento_ajustado.loc[df_orcamento_ajustado['VALOR_DETALHAMENTO_AÇÃO'].isna(), 'VALOR_DETALHAMENTO_AÇÃO'] = 0

df_orcamento_ajustado.loc[:, 'Vl_Liquidado_N_Detalhado'] = (
    df_orcamento_ajustado.loc[:, 'Vl_Liquidado']
    - df_orcamento_ajustado.loc[:, 'VALOR_DETALHAMENTO_AÇÃO']).round(2)

df_orcamento_ajustado

In [ ]:
df_orcamento_ajustado[df_orcamento_ajustado['Vl_Liquidado_N_Detalhado']<0]

In [ ]:
df_orcamento_ajustado[['Vl_Liquidado', 'VALOR_DETALHAMENTO_AÇÃO']].sum()

Agora, vamos adicionar os dados não detalhados ao dataframe que contém o orçamento detalhado.

In [ ]:
orcamento_cols_map = {'Cd_Orgao': 'CÓDIGO_ÓRGÃO',
                      'Sigla_Orgao': 'SIGLA_ÓRGÃO',
                      'Ds_Orgao': 'DESCRIÇÃO_ÓRGÃO',
                      'Cd_Programa': 'CÓDIGO_PROGRAMA',
                      'Ds_Programa': 'DESCRIÇÃO_PROGRAMA',
                      'ProjetoAtividade': 'CÓDIGO_PROJ_ATIV',
                      'Ds_Projeto_Atividade': 'DESCRIÇÃO_PROJ_ATIV',
                      'Vl_Liquidado_N_Detalhado': 'Vl_Liquidado'}

df_orcamento_ajustado = (
    df_orcamento_ajustado
    .drop(columns=['VALOR_DETALHAMENTO_AÇÃO', 'Vl_Liquidado'])
    .rename(columns=orcamento_cols_map)
    )

df_orcamento_ajustado

In [ ]:
df_orcamento_r = (df_orcamento_r
                  .rename(columns={'VALOR_DETALHAMENTO_AÇÃO': 'Vl_Liquidado'}))

df_orcamento_r

In [ ]:
df_orcamento_final = pd.concat([df_orcamento_r, df_orcamento_ajustado])

df_orcamento_final

In [ ]:
df_orcamento_final.loc[df_orcamento_final['Vl_Orcado_Ano'].isna(), 'Vl_Orcado_Ano'] = 0
df_orcamento_final.loc[df_orcamento_final['Vl_Orcado_Atualizado'].isna(), 'Vl_Orcado_Atualizado'] = 0
df_orcamento_final.loc[df_orcamento_final['NIVEL_REGIONALIZAÇÃO'].isna(), 'NIVEL_REGIONALIZAÇÃO'] = 'Não detalhado'

df_orcamento_final

In [ ]:
subs_orcamento = (
    df_orcamento_final.loc[~df_orcamento_final['SUBPREFEITURA'].isna(), 'SUBPREFEITURA']
    .apply(unidecode)
    .unique()
    .tolist()
)

subs_orcamento.sort()

subs_orcamento

In [ ]:
subs_orcamento[:-6]

In [ ]:
len(subs_orcamento[:-6])

In [56]:
subs_qlik_orcamento = subs_qlik.copy()
# As 3 subs abaixo não aparecem na lista de liquidação do orçamento
subs_qlik_orcamento.remove('ERMELINO MATARAZZO')
subs_qlik_orcamento.remove('PIRITUBA-JARAGUA')
subs_qlik_orcamento.remove('VILA MARIANA')
# Guainases e Vila Prudente aparecem em uma ordenação diferente, por isso serão
# removidas e adicionadas novamente ao final da lista
subs_qlik_orcamento.remove('GUAIANASES')
subs_qlik_orcamento.remove('VILA PRUDENTE')
subs_qlik_orcamento.append('GUAIANASES')
subs_qlik_orcamento.append('VILA PRUDENTE')

In [ ]:
mapper_orcamento = {
    so: sq
    for so, sq in zip(subs_orcamento, subs_qlik_orcamento)
}

mapper_orcamento

In [ ]:
df_orcamento_final.insert(
    7,
    'sub.NOME',
    df_orcamento_final.loc[:,'SUBPREFEITURA'].apply(lambda s: unidecode(s) if isinstance(s, str) else None).map(mapper_orcamento)
)

df_orcamento_final

In [ ]:
df_orcamento_final.loc[:, 'ano'] = 2024

df_orcamento_final = df_orcamento_final.merge(df_subs_ano,
                              how='left',
                              on=['sub.NOME', 'ano'])

df_orcamento_final

# Armazenamento

Finalmente, vamos exportar os dados em formato csv compatível com o Qlik e no padrão do excel para português do Brasil.

In [60]:
base_path = path.join('data_output', 'urbanismo')

if not path.exists(base_path):
    makedirs(base_path)

for name, df in [('orcamento-habitacao', df_orcamento_final),
                 ('producao-his', df_his),
                 ('pdm-meta-12', df_meta_12),
                 ('emissoes-tpu', df_tpu),
                 ('subprefeitura-ano', df_subs_ano)]:
    
    filepath = path.join(base_path, f'{name}.csv')

    df.to_csv(filepath,
              index=False,
              sep=';',
              decimal=',',
              encoding='latin1')